<a href="https://colab.research.google.com/github/Nadirjohar24/question_answer_nadir/blob/main/question__answering_nadir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install PyPDF2
!pip install pdfminer.six
!pip install simpletransformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 13.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 100.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 90.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [5]:
import os
import json
import numpy as np
import pandas as pd
import PyPDF2
import pdfminer
from simpletransformers.question_answering import QuestionAnsweringModel


In [14]:
with open(r'train-v2.0.json', 'r') as f:
    squad_train = json.load(f)
    



In [15]:
with open(r'dev-v2.0.json', 'r') as f:
    squad_dev = json.load(f)

In [29]:
def convert_to_df(data):
    contexts = []
    questions = []
    answers = []
    for article in data['data']:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                for answer in qa['answers']:
                    contexts.append(paragraph['context'])
                    questions.append(qa['question'])
                    answers.append(answer['text'])

    df = pd.DataFrame({'context': contexts, 'question': questions, 'answer': answers})
    return df

train_df = squad_train
train_df = [item for topic in train_df['data'] for item in topic['paragraphs'] ]

train_df = train_df[:50]
dev_df = squad_dev
dev_df = [item for topic in dev_df['data'] for item in topic['paragraphs'] ]

dev_df = dev_df[:50]


In [41]:
def load_pdf(file_path):
    with open('cars.pdf', 'rb') as f:
        pdf_reader = PyPDF2.PdfReader(f)
        text = ''
        for i in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[i]
            text += page.extract_text()
        return text

context =load_pdf('cars.pdf')


In [31]:
model = QuestionAnsweringModel('bert', 'bert-base-cased', args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 2}, use_cuda=False)

model.train_model(train_df, eval_df=dev_df)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/76 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/76 [00:00<?, ?it/s]

(152, 2.2877847015073427)

In [36]:
import json
with open(r"test.json", "r") as read_file:
    test = json.load(read_file)

In [37]:
result, texts = model.eval_model(test)

add example index and unique id: 100%|██████████| 3/3 [00:00<00:00, 23741.34it/s]


Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [50]:
question = 'societal benefits of cars?'
##answer = model.predict([{'context': context, 'qas': [{'question': question}]}])[0]['answer']
answer = model.predict([{'context': context, 'qas': [{'question': question,'id':'0',}]}])[0]

print(answer)


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 9597.95it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['Traffic collisions are the largest cause of injury -related deaths worldwide .[13] Personal benefits include on -dema nd transportation, mobility', 'Traffic collisions are the largest cause of injury -related deaths worldwide .[13] Personal benefits include on -dema nd transportation, mobility, independence, and convenience .[14] Societal benefits include economic benefits, such as job and wealth creation from the automotive industry , transportation provision, societal well -being', '', 'society include maintaining roads, land use, road congestion , air pollution , noise pollution , public health , and disposing of the vehicle at the end of its life . Traffic collisions are the largest cause of injury -related deaths worldwide .[13] Personal benefits include on -dema nd transportation, mobility', 'society include maintaining roads, land use, road congestion , air pollution , noise pollution , public health , and disposing of the vehicle at the end of its life 